In [1]:
%%shell
pip install -q https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
pip install -q tqdm==4.56.0
pip install -q apyori

     \ 39.5MB 874kB/s
     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 604kB 8.9MB/s 
     |████████████████████████████████| 3.1MB 14.1MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 81kB 288kB/s 
     |████████████████████████████████| 296kB 22.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 81kB 3.3MB/s 


In [2]:
!shred -u setup_colab_general.py
!wget -q "https://github.com/jpcano1/python_utils/raw/main/setup_colab_general.py" -O setup_colab_general.py
import setup_colab_general as setup_general
setup_general.setup_general()

shred: setup_colab_general.py: failed to open for writing: No such file or directory


  0%|          | 0/3 [00:00<?, ?KB/s]

General Functions Enabled Successfully


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder

import matplotlib.pyplot as plt
plt.style.use("seaborn-deep")
import seaborn as sns

import pandas_profiling

import utils.general as gen

from apyori import apriori as ap
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
dataset_id = "16dXuaaAXAWSHyfDUG9qf0VFDXGPV7o6m"

In [5]:
gen.download_file_from_google_drive(dataset_id, "employee.csv", size=227)

  0%|          | 0/227 [00:00<?, ?KB/s]

In [6]:
data_dir = gen.create_and_verify("data", "employee.csv")

In [7]:
data = pd.read_csv(data_dir)

In [8]:
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [20]:
profile = pandas_profiling.ProfileReport(data, minimal=True)

In [21]:
profile.to_file("profile.html")

Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
data.drop(
    columns=["Over18", "EmployeeCount", "StandardHours"],
    inplace=True
)

In [10]:
for col in data:
    dtype = data[col].dtype
    data[col] = data[col].astype("int16") if dtype == "int64" else data[col].astype("category")

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   Age                       1470 non-null   int16   
 1   Attrition                 1470 non-null   category
 2   BusinessTravel            1470 non-null   category
 3   DailyRate                 1470 non-null   int16   
 4   Department                1470 non-null   category
 5   DistanceFromHome          1470 non-null   int16   
 6   Education                 1470 non-null   int16   
 7   EducationField            1470 non-null   category
 8   EmployeeNumber            1470 non-null   int16   
 9   EnvironmentSatisfaction   1470 non-null   int16   
 10  Gender                    1470 non-null   category
 11  HourlyRate                1470 non-null   int16   
 12  JobInvolvement            1470 non-null   int16   
 13  JobLevel                  1470 non-null   int16 

In [12]:
for col_name, col_data in data.iteritems():
    print(f"{col_name} has {col_data.isnull().sum() / len(col_data)}% of null data")

Age has 0.0% of null data
Attrition has 0.0% of null data
BusinessTravel has 0.0% of null data
DailyRate has 0.0% of null data
Department has 0.0% of null data
DistanceFromHome has 0.0% of null data
Education has 0.0% of null data
EducationField has 0.0% of null data
EmployeeNumber has 0.0% of null data
EnvironmentSatisfaction has 0.0% of null data
Gender has 0.0% of null data
HourlyRate has 0.0% of null data
JobInvolvement has 0.0% of null data
JobLevel has 0.0% of null data
JobRole has 0.0% of null data
JobSatisfaction has 0.0% of null data
MaritalStatus has 0.0% of null data
MonthlyIncome has 0.0% of null data
MonthlyRate has 0.0% of null data
NumCompaniesWorked has 0.0% of null data
OverTime has 0.0% of null data
PercentSalaryHike has 0.0% of null data
PerformanceRating has 0.0% of null data
RelationshipSatisfaction has 0.0% of null data
StockOptionLevel has 0.0% of null data
TotalWorkingYears has 0.0% of null data
TrainingTimesLastYear has 0.0% of null data
WorkLifeBalance has 0.0

In [13]:
f"{data.duplicated().sum() / len(data)}% of duplicated data"

'0.0% of duplicated data'

In [14]:
label_cols = [
    "BusinessTravel", "Department",
    "EducationField", "Gender", "JobRole",
    "MaritalStatus"
]

binary_cols = [
    "Attrition", "OverTime"
]

In [15]:
label_encoder = LabelEncoder()
binary_encoder = LabelBinarizer()

In [16]:
for col in binary_cols:
    data[col] = binary_encoder.fit_transform(data[col]).astype("int16")

In [17]:
for col in label_cols:
    data[col] = label_encoder.fit_transform(data[col]).astype("int16")

In [18]:
data.drop(columns=["EmployeeNumber"], inplace=True)

In [19]:
numerical_cols = [
    "Age", "DailyRate", "DistanceFromHome", "Education",
    "EducationField", "JobRole", 
    "HourlyRate", "MonthlyIncome", "MonthlyRate", 
    "NumCompaniesWorked", "PercentSalaryHike", 
    "PerformanceRating", "TotalWorkingYears", 
    "TotalWorkingYears", "TrainingTimesLastYear", 
    "YearsAtCompany", "YearsInCurrentRole", "YearsSinceLastPromotion", 
    "YearsWithCurrManager"
]

In [20]:
data.drop(columns=numerical_cols, inplace=True)

In [21]:
encoder = OneHotEncoder()

In [22]:
data_one_hot = encoder.fit_transform(data).toarray()

In [23]:
full_data = {}

In [24]:
counter = 0
for idx, col in enumerate(data):
    for cat in encoder.categories_[idx]:
        full_data[f"{col}_{cat}"] = data_one_hot[:, counter]
        counter += 1
full_data = pd.DataFrame(full_data, dtype="uint8")

In [25]:
full_data

,Attrition_0,Attrition_1,BusinessTravel_0,BusinessTravel_1,BusinessTravel_2,Department_0,Department_1,Department_2,EnvironmentSatisfaction_1,EnvironmentSatisfaction_2,EnvironmentSatisfaction_3,EnvironmentSatisfaction_4,Gender_0,Gender_1,JobInvolvement_1,JobInvolvement_2,JobInvolvement_3,JobInvolvement_4,JobLevel_1,JobLevel_2,JobLevel_3,JobLevel_4,JobLevel_5,JobSatisfaction_1,JobSatisfaction_2,JobSatisfaction_3,JobSatisfaction_4,MaritalStatus_0,MaritalStatus_1,MaritalStatus_2,OverTime_0,OverTime_1,RelationshipSatisfaction_1,RelationshipSatisfaction_2,RelationshipSatisfaction_3,RelationshipSatisfaction_4,StockOptionLevel_0,StockOptionLevel_1,StockOptionLevel_2,StockOptionLevel_3,WorkLifeBalance_1,WorkLifeBalance_2,WorkLifeBalance_3,WorkLifeBalance_4
0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0
1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
2,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0
3,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0
4,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
1466,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0
1467,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
1468,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0


In [27]:
frequent_itemsets = apriori(full_data, min_support=0.07, use_colnames=True)

In [28]:
%%time
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)

CPU times: user 65.3 ms, sys: 1.77 ms, total: 67.1 ms
Wall time: 74.2 ms


In [31]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(JobLevel_1, StockOptionLevel_0)","(MaritalStatus_2, Department_1)",0.174830,0.206803,0.109524,0.626459,3.029260,0.073369,2.123455
1,"(MaritalStatus_2, Department_1)","(JobLevel_1, StockOptionLevel_0)",0.206803,0.174830,0.109524,0.529605,3.029260,0.073369,1.754208
2,"(JobLevel_1, Attrition_0, MaritalStatus_2)","(Department_1, StockOptionLevel_0)",0.084354,0.276871,0.071429,0.846774,3.058374,0.048073,4.719370
3,"(JobLevel_1, Attrition_0, StockOptionLevel_0)","(Department_1, MaritalStatus_2)",0.110204,0.206803,0.071429,0.648148,3.134137,0.048638,2.254350
6,"(OverTime_0, StockOptionLevel_0, JobSatisfacti...","(Attrition_0, MaritalStatus_2)",0.091156,0.238095,0.070068,0.768657,3.228358,0.048364,3.293395
8,"(BusinessTravel_2, Department_1, MaritalStatus_2)","(JobLevel_1, StockOptionLevel_0)",0.144898,0.174830,0.077551,0.535211,3.061325,0.052219,1.775366
10,"(OverTime_0, JobLevel_1, StockOptionLevel_0)","(Department_1, MaritalStatus_2)",0.123810,0.206803,0.078912,0.637363,3.081984,0.053307,2.187302
12,"(Gender_1, Attrition_0, MaritalStatus_2)","(OverTime_0, WorkLifeBalance_3, StockOptionLev...",0.134694,0.187075,0.078912,0.585859,3.131680,0.053714,1.962917


In [32]:
for idx, data in rules.iterrows():
    antecedents	= ", ".join(x for x in data["antecedents"])
    consequents = ", ".join(x for x in data["consequents"])
    support = data["support"]
    confidence = data["confidence"]
    lift = data["lift"]

    print(f"Rule: {antecedents} ===> {consequents}")
    print(f"Confidence: {confidence}")
    print(f"Lift: {lift}")
    print(f"Support: {support}")
    print("========================================")

Rule: JobLevel_1, StockOptionLevel_0 ===> MaritalStatus_2, Department_1
Confidence: 0.6264591439688716
Lift: 3.0292596764284254
Support: 0.10952380952380952
Rule: MaritalStatus_2, Department_1 ===> JobLevel_1, StockOptionLevel_0
Confidence: 0.5296052631578948
Lift: 3.0292596764284254
Support: 0.10952380952380952
Rule: JobLevel_1, Attrition_0, MaritalStatus_2 ===> Department_1, StockOptionLevel_0
Confidence: 0.8467741935483871
Lift: 3.058373622889752
Support: 0.07142857142857142
Rule: JobLevel_1, Attrition_0, StockOptionLevel_0 ===> Department_1, MaritalStatus_2
Confidence: 0.6481481481481481
Lift: 3.134137426900585
Support: 0.07142857142857142
Rule: OverTime_0, StockOptionLevel_0, JobSatisfaction_4 ===> Attrition_0, MaritalStatus_2
Confidence: 0.7686567164179104
Lift: 3.228358208955224
Support: 0.07006802721088436
Rule: BusinessTravel_2, Department_1, MaritalStatus_2 ===> JobLevel_1, StockOptionLevel_0
Confidence: 0.5352112676056339
Lift: 3.0613251493396176
Support: 0.07755102040816327